In [53]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By # 웹사이트의 구성요소를 선택하기 위해 By 모듈을 불려옵니다
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지가 전부 로드될때까지 기다리는 (Explicitly wait) 기능을 하는 모듈입니다
from webdriver_manager.chrome import ChromeDriverManager # 크롬에서 크롤링을 하기 위해, 웹 드라이버를 설치하는 모듈입니다
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC # 크롬의 어떤 부분의 상태를 확인하는 모듈입니다
import time # 정해진 시간만큼 기다리게 하기 위한 패키지입니다
from selenium.webdriver.common.keys import Keys

In [54]:
Tsherpa_url= "https://edu.tsherpa.co.kr/LectureInfo/LectureReviews2"


In [55]:
driver = webdriver.Chrome()
driver.get(Tsherpa_url)

In [56]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mainForm"]'))
    )
finally:
    pass

In [57]:
fnm = '' # 맨 첫번째 상호 이름입니다. 다음 페이지에서도 똑같은 상호가 나온다면, 다음 페이지가 없다고 인식하고, brk 값이 바뀌면서 반복문이 종료되지요
brk = 1
nxt = 1
res = [] # 결과 파일은 판다스 데이터프레임으로 입력할겁니다
page_count = 1


while True:
    
    for i in range(1,11):
        
        time.sleep(0.5)
        hour = driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a/span').text
        
        if driver.find_element(By.XPATH, f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a').is_displayed():
            driver.find_element(By.XPATH, f'//*[@id="mainForm"]/table/tbody/tr[{i}]/td[2]/a').send_keys(Keys.ENTER)
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[-1])

            nm = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[1]/td')
            yr = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[2]')
            scp = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[1]')
            desc = driver.find_elements(By.XPATH, '//*[@id="mainForm"]/div[2]/table/tbody/tr[3]/td')

        res.append([nm[0].text, hour, yr[0].text.split('|')[1].strip(), scp[0].text, desc[0].text])

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    page_count += 1
    brk += 1
    
    # 다음 페이지로 넘어가는 코드입니다    
    if page_count+nxt != 11:
        driver.find_element(By.XPATH, f'//*[@id="mainForm"]/div[3]/a[{page_count+nxt}]').send_keys(Keys.ENTER)
    else:
       driver.find_element(By.XPATH, f'//*[@id="mainForm"]/div[3]/a[12]').send_keys(Keys.ENTER)
       page_count = 1

    time.sleep(25)

    if brk == 25:
        driver.quit()
        break


            





In [58]:
df = pd.DataFrame(res)
df.to_csv('./res_Tsherpa.csv', index=False) # 데이터가 실시간으로 저장되도록 합니다
df

,0,1,2,3,4
0,손그림 파워포인트 콘텐츠 만들기,30시간,2023.10.23,,파워포인트 주로 사용하는데 매일 하던거만 하다 새로운걸 알아가네요~\n감사합니다~
1,손그림그리기 잼있는 연수,15시간,2023.10.22,,도형을 이용해 간단하게 손그림 그리는 방법을 배우는 유익한 연수였어요..\n그런데 ...
2,똥손 선생님을 위한 최고의 선택!,15시간,2023.10.19,,저는 미술에 정말 소질이 없는 사람입니다.. 그래서 아이들과 미술을 하는 것도 힘들...
3,내 생애 최고의 독서수업 방법 연수,30시간,2023.10.19,,교직 생활이 연수가 늘어갈수록 다양한 독서 연수 강좌를 들어왔습니다. 이번 연수는 ...
4,생각이 행동으로 연결되는 연수였습니다. 강추!!!,30시간,2023.10.18,,"근래 들어 정말 기분좋게, 즐겁게 들었던 연수입니다.\n환경인가? 하며 들었는데 다..."
...,...,...,...,...,...
235,나의 평화가 가능한 이유,30시간,2023.06.09,,역사시간에 배운 유명한 독립운동가뿐 아니라 알려지지 않았던 독립운동을 하신 분들의 ...
236,핵심 표현들을 활용한 회화수업!!,30시간,2023.06.09,,핵심단어 30개를 가지고 여기저기 다 활용하여 완성하는 영어회화 수업입니다. 강추입니다.
237,연수가 재미있어요! 강추합니다.,15시간,2023.06.09,,막연하게 생각했던 남학생들의 지도가 이렇게 명료하게 정리되어서 연수가 참 좋았습니다...
238,놀이의 모습이 그려지는 연수,30시간,2023.06.08,,놀이는 언제 들어도 재미있고 활용도가 높습니다. 오프라인 연수가 제일 효과가 좋지만...
